# Model selection
- train val split
- explore model
- select a good one and do tuning
- train and save to model directory

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Read processed data

In [4]:
from data.read import read_processed_data
df = read_processed_data('../data/processed/train.pkl')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329780 entries, 0 to 1329779
Data columns (total 9 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   trip_distance                1329780 non-null  float64
 1   pickup_datetime_date         1329780 non-null  float64
 2   pickup_datetime_day_of_week  1329780 non-null  float64
 3   pickup_datetime_hour         1329780 non-null  float64
 4   pickup_latitude              1329780 non-null  float64
 5   pickup_longitude             1329780 non-null  float64
 6   dropoff_latitude             1329780 non-null  float64
 7   dropoff_longitude            1329780 non-null  float64
 8   log_trip_duration            1329780 non-null  float64
dtypes: float64(9)
memory usage: 91.3 MB


# Train val split

In [7]:
X = df.drop(['log_trip_duration'], axis=1)
y = df['log_trip_duration']

In [35]:
# without column names
# np.array(X).shape
# y.ravel()

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [15]:
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(1196802, 8) (132978, 8)
(1196802,) (132978,)


# Explore models
- evaluate by rmse

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
def evaluate_regressor_skl(regressor, X_train, y_train, X_val, y_val):
    '''
    for sklearn models
    '''
    regressor.fit(X_train, y_train)
    # make predictions on training data
    y_pred_train = regressor.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))

    # make predictions on test data
    y_pred_val = regressor.predict(X_val)
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

    print(f'Predicting with {regressor}')
    print('Training error:')
    print(f'Root mean squared error = {rmse_train}')
    
    print('Validation error:')
    print(f'Root mean squared error = {rmse_val}')

    return regressor, y_pred_train, y_pred_val

In [18]:
# try HistogramGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

hgb_results = evaluate_regressor_skl(
    HistGradientBoostingRegressor(random_state=42),
    X_train, y_train,
    X_val, y_val
    )

Predicting with HistGradientBoostingRegressor(random_state=42)
Training error:
Root mean squared error = 0.38732930810657246
Validation error:
Root mean squared error = 0.3946265400959257


# Cross validation

In [19]:
from sklearn.model_selection import cross_val_score

cv_scores_hgb = np.sqrt(-1*np.array(cross_val_score(HistGradientBoostingRegressor(random_state=42), X, y, scoring='neg_mean_squared_error', cv=5)))

In [20]:
print(cv_scores_hgb)
print(f'Mean = {np.mean(cv_scores_hgb)}')

[0.3913187  0.39114067 0.39033772 0.39144576 0.38939521]
Mean = 0.39072761412687507


# Hyperparams tuning

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    # param_name: [to random search] + [default value]
    'max_leaf_nodes': [31, 150, 300]
}

regressor = GridSearchCV(HistGradientBoostingRegressor(random_state=42), param_grid, scoring='neg_root_mean_squared_error', cv=3, refit=True)
search = regressor.fit(X, y)

In [22]:
# we got better results with more max_leaf_nodes
search.cv_results_

{'mean_fit_time': array([19.53243907, 24.09316913, 34.82859087]),
 'std_fit_time': array([1.43024878, 0.13117922, 3.04362348]),
 'mean_score_time': array([1.86253564, 1.72625391, 2.16300265]),
 'std_score_time': array([0.31782028, 0.04289972, 0.36191248]),
 'param_max_leaf_nodes': masked_array(data=[31, 150, 300],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_leaf_nodes': 31},
  {'max_leaf_nodes': 150},
  {'max_leaf_nodes': 300}],
 'split0_test_score': array([-0.39124732, -0.37303398, -0.36574581]),
 'split1_test_score': array([-0.38992782, -0.37205564, -0.36575616]),
 'split2_test_score': array([-0.39015086, -0.37197654, -0.36521196]),
 'mean_test_score': array([-0.390442  , -0.37235539, -0.36557131]),
 'std_test_score': array([0.00057668, 0.00048092, 0.00025414]),
 'rank_test_score': array([3, 2, 1])}

In [23]:
# save model to model directory
from joblib import dump

dump(search.best_estimator_, '../models/best_estimator.joblib')

['../model/best_estimator.joblib']

In [10]:
# test we can load and use for making predictions
from joblib import load
from data.read import read_processed_test_data
from models.predict import make_predictions

loaded_model = load('../models/best_estimator.joblib')

test = read_processed_test_data('../data/processed/test.pkl')

,trip_distance,pickup_datetime_date,pickup_datetime_day_of_week,pickup_datetime_hour,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,-0.007877,1.667006,-0.025000,1.467616,-0.943026,-0.405850,0.129094,-0.572888
1,-0.002395,1.667006,-0.025000,1.467616,-3.244683,0.575647,-3.647017,0.762413
2,-0.624152,1.667006,-0.025000,1.467616,-0.697354,-0.787682,-0.883454,-0.396414
3,1.071540,1.667006,-0.025000,1.467616,0.820549,0.909281,-0.848180,-0.408156
4,-0.771907,1.667006,-0.025000,1.467616,0.359407,0.329021,0.099652,0.687596
...,...,...,...,...,...,...,...,...
625129,-0.769085,-1.665979,0.487731,-2.139157,-1.249272,-1.034934,-0.729843,-1.060038
625130,1.468301,-1.665979,0.487731,-2.139157,-0.423166,-1.153865,1.090581,1.043899
625131,2.305839,-1.665979,0.487731,-2.139157,0.288203,0.244830,-0.169735,4.421405
625132,7.537640,-1.665979,0.487731,-2.139157,-0.875196,0.071128,5.166660,5.403757


In [9]:
# make_predictions(loaded_model, test)
y_test = pd.read_pickle('../models/predictions.pkl')['trip_duration']

y_test

0          790.801020
1          609.325818
2          405.487914
3          972.701657
4          342.239061
             ...     
625129     290.722729
625130    1229.820692
625131    1489.770467
625132    1506.953816
625133    1150.844801
Name: trip_duration, Length: 625134, dtype: float64

In [11]:
from data.read import read_test_data

submission = pd.DataFrame()
submission['id'] = read_test_data('../data/raw/test.csv')['id']
submission['trip_duration'] = pd.Series(y_test)

submission

,id,trip_duration
0,id3004672,790.801020
1,id3505355,609.325818
2,id1217141,405.487914
3,id2150126,972.701657
4,id1598245,342.239061
...,...,...
625129,id3008929,290.722729
625130,id3700764,1229.820692
625131,id2568735,1489.770467
625132,id1384355,1506.953816


In [12]:
submission.to_csv('../models/submission.csv', index=False)